In [ ]:
# Practice activity: Implementing an ML model from scratch
# Dataset: CIFAR-10 dataset, a well-known dataset consisting of 60,000 32x32 color images in 10 different classes. The dataset is already split into 50,000 training images and 10,000 test images.
# Choosing a framework: PyTorch, a popular deep learning framework known for its flexibility and ease of use.
# Set up your development environment
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load the CIFAR-10 dataset
transform = transforms.Compose( # Define the transformations to be applied to the images
    [transforms.ToTensor(), # Convert images to PyTorch tensors
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normalize the images
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True, transform=transform) # Download and load the training dataset
trainloader = DataLoader(trainset, batch_size=100,
                         shuffle=True, num_workers=2) # Create a DataLoader for the training dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform) # Download and load the test dataset
testloader = DataLoader(testset, batch_size=100,
                        shuffle=False, num_workers=2) # Create a DataLoader for the test dataset
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  # Define the class labels

# Build the model. Define your model architecture using PyTorch's nn.Module
import torch.nn.functional as F
class Net(nn.Module): # Define a convolutional neural network
    def __init__(self): # Initialize the layers of the network
        super(Net, self).__init__() # Call the constructor of the parent class
        self.conv1 = nn.Conv2d(3, 32, 5) # First convolutional layer
        self.pool = nn.MaxPool2d(2, 2) # Max pooling layer
        self.conv2 = nn.Conv2d(32, 64, 5) # Second convolutional layer
        self.fc1 = nn.Linear(64 * 5 * 5, 128) # First fully connected layer
        self.fc2 = nn.Linear(128, 10) # Second fully connected layer

    def forward(self, x): # Define the forward pass
        x = self.pool(F.relu(self.conv1(x))) # Apply first convolution, ReLU activation, and pooling
        x = self.pool(F.relu(self.conv2(x))) # Apply second convolution, ReLU activation, and pooling
        x = x.view(-1, 64 * 5 * 5) # Flatten the tensor
        x = F.relu(self.fc1(x)) # Apply ReLU activation
        x = self.fc2(x) # Apply second fully connected layer
        return x # Return the output

model = Net() # Instantiate the model

# Model training. Define the loss function and optimizer, then train the model
criterion = nn.CrossEntropyLoss() # Define the loss function
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # Define the optimizer

for epoch in range(10): # Loop over the dataset multiple times
    running_loss = 0.0 # Initialize running loss for the epoch
    for i, data in enumerate(trainloader, 0): # Iterate over the training data
        inputs, labels = data # Get the inputs and labels
        optimizer.zero_grad() # Zero the parameter gradients
        outputs = model(inputs) # Forward pass
        loss = criterion(outputs, labels) # Compute the loss
        loss.backward() # Backward pass
        optimizer.step() # Optimize the model
        running_loss += loss.item() # Accumulate the loss
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}")
            running_loss = 0.0 # Reset running loss

print("Finished Training") # Indicate that training is complete

# Model evaluation. Evaluate the model's performance on the test dataset
correct = 0 # Initialize the count of correct predictions
total = 0 # Initialize the total count of predictions
with torch.no_grad(): # Disable gradient calculation for evaluation
    for data in testloader: # Iterate over the test data
        images, labels = data # Get the images and labels
        outputs = model(images) # Forward pass
        _, predicted = torch.max(outputs.data, 1) # Get the predicted class
        total += labels.size(0) # Update the total count
        correct += (predicted == labels).sum().item() # Update the count of correct predictions

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%") # Print the accuracy

# Visualize some predictions
dataiter = iter(testloader) # Create an iterator for the test data
images, labels = next(dataiter) # Get a batch of test images and labels
outputs = model(images) # Forward pass
_, predicted = torch.max(outputs, 1) # Get the predicted classes

# Function to show an image
def imshow(img):
    img = img / 2 + 0.5     # Unnormalize the image
    npimg = img.numpy() # Convert the tensor to a NumPy array
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # Display the image
    plt.show() # Show the plot
imshow(torchvision.utils.make_grid(images)) # Show a grid of test images
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]}' for j in range(4))) # Print the ground truth labels
print('Predicted: ', ' '.join(f'{classes[predicted[j]]}' for j in range(4))) # Print the predicted labels

# Save the model
torch.save(model.state_dict(), 'cifar10_model.pth') # Save the model's state dictionary
print("Model saved as cifar10_model.pth") # Indicate that the model has been saved

In [ ]:
# Load the model (for future use)
model = Net() # Instantiate the model
model.load_state_dict(torch.load('cifar10_model.pth')) # Load the saved state dictionary
model.eval() # Set the model to evaluation mode
print("Model loaded from cifar10_model.pth") # Indicate that the model has been loaded